In [13]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
import librosa
import librosa.display
from scipy import signal
%matplotlib inline

In [2]:
# Import data
train_csv = pd.read_csv("data/train.csv")

In [4]:
print("There are {:,} unique bird species in the dataset.".format(len(train_csv['species'].unique())))

There are 264 unique bird species in the dataset.


In [22]:
# Audio data loading
path = "data/train_audio/"

directory_contents = os.listdir(path)
directory_contents.sort()

train_data = []
train_labels = []
for i in range(3):
    curr_bird_dir = path + directory_contents[i] + "/"
    recording_names = os.listdir(curr_bird_dir)
    for filename in recording_names:
        sample, sr = librosa.load(curr_bird_dir+filename)
        # pass through fourier transform
        f, t, spec = signal.stft(sample, sr)
        print(spec.shape)
        train_data.append(spec)
        train_labels += [i]

/opt/anaconda3/lib/python3.7/site-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')


(129, 10139)


/opt/anaconda3/lib/python3.7/site-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')


(129, 917)


/opt/anaconda3/lib/python3.7/site-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')


...

(129, 3754)


/opt/anaconda3/lib/python3.7/site-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')


(129, 5819)


/opt/anaconda3/lib/python3.7/site-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')


KeyboardInterrupt: 

In [19]:
train_data_yeet = np.asarray(train_data)
train_labels = np.asarray(train_labels)

ValueError: could not broadcast input array from shape (129,10139) into shape (129)

In [20]:
print(train_labels.shape())
print(train_data_yeet.shape())

AttributeError: 'list' object has no attribute 'shape'